In [1]:
import os
import sys

print(os.path.abspath(os.curdir))

import traceback
from dipy.tracking import utils
import nibabel as nib
import numpy as np
from nibabel import trackvis
from responsestuff.responsestuff.autoresponse import fskeys
import scipy
import json

/Users/ashley.nguyen27/structural-network-generation
You do not have Matplotlib installed. Some visualization functions might not work for you.


In [2]:
ctx_array = fskeys['ctx'] # labels that belong to the cerebral cortex
deep_grey_array = fskeys['deep gray'] # labels that belong to grey matter

In [18]:
config_path = open("config.json")
config_dict = dict(json.load(config_path))

subjid = config_dict["subjid"]
track_dir = config_dict["track_dir"]
label_dir = config_dict["label_dir"]
streamline_file = "{}/{}/prob-localtrack-clean.trk".format(track_dir, subjid)
label_file = "{}/{}/aseg_to_fa_aff.nii.gz".format(label_dir, subjid)

save_dir = config_dict["save_dir"]
save_file = "{}/{}/network/metric_{{}}_mat.csv".format(save_dir, subjid)

In [19]:
trk, hdr = trackvis.read(streamline_file)

In [ ]:
streamlines = [i[0] for i in trk] # "strips" streamlines (only extracts points)
trk_save = [(i, None, None) for i in streamlines] # unstripped version to save if needed
assert str(hdr['voxel_order']).lower() == "las" # NOT SURE?
vs = hdr['voxel_size'] # NOT SURE?
aff = np.diag([vs[0], vs[1], vs[2], 1]) # NOT SURE?
aff[:, 3] = np.dot(aff, [.5, .5, .5, 1]) # NOT SURE?

In [21]:
md_file = "{}/{}/data_{}.nii.gz".format(label_dir, subjid, "md")
fa_file = "{}/{}/data_{}.nii.gz".format(label_dir, subjid, "fa")
vol_md = nib.load(md_file).get_data()
vol_fa = nib.load(fa_file).get_data()

FileNotFoundError: No such file or no access: '/projects/ps-henrylab/hcp/hcp_structural/mtdModelFit/100206/data_md.nii.gz'

In [22]:
aseg = nib.load(label_file).get_data()
gray_region = (np.in1d(aseg, fskeys['ctx']) |
               np.in1d(aseg, fskeys['deep gray']))
gray_region.shape = aseg.shape # "unlinearize": convert linear to 3D shape

In [23]:
aseg_grey = aseg * gray_region # mask all non-grey matter regions in this mapping

thalamus_mask = np.logical_or(aseg_grey == 10,aseg_grey == 49) 
visual_cortex_mask = np.logical_or(aseg_grey == 2011,aseg_grey == 1011)
optic_mask = np.logical_or(thalamus_mask, visual_cortex_mask)
aseg_optic = aseg_grey * optic_mask
labels, lt = utils.reduce_labels(aseg_optic)
# labels = the new 3D image with the reduced labels starting from 0,1,2,....
# lt -> list with mapping to old labels (i.e. new label 1 has old label lt[1])

# please see link below for regional lookup table
# https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/AnatomicalROI/FreeSurferColorLUT

non_lh_ctx_regions = []
lh_ctx_regions = []

non_rh_ctx_regions = []
rh_ctx_regions = []
atlas_regions = []

def get_reduced_map(desired_regions):
    pass


In [ ]:
mat, mapping = utils.connectivity_matrix(streamlines, labels,
                                             affine=aff,
                                             return_mapping=True,
                                             mapping_as_streamlines=True)
left_optic_rad = mapping[1,3] + mapping[1,4]
right_optic_rad = mapping[2,3] + mapping[2,4]
optic_rads = left_optic_rad + right_optic_rad
optic_rad_unstripped = [(i, None, None) for i in optic_rads]

In [ ]:
streamline_save_dir = "{}/{}/network/connectiveatlas_{}.trk".format(save_dir, subjid,subjid)
trackvis.write(open(streamline_save_dir,'wb'), optic_rad_unstripped, hdr)